# Semantic Redundancy Measurements on Realistic Prompts

> **Note:** You can switch between TF-IDF and transformer embeddings by running `python scripts/semantic_redundancy_metric.py --backend {tfidf,transformer}`. This notebook defaults to TF-IDF for portability; swap in the transformer encoder if the dependency is available.

This notebook approximates the mutual-information-style redundancy ratio $\rho$ by comparing prompt sentences to the reference answer using contextual embeddings. We:

1. Define a small factual QA set (question, answer, and several prompt variants with noisy or conflicting context).
2. Encode sentences (TF-IDF by default) and compute cosine similarity between each sentence and the ground-truth answer embedding.
3. Treat the fraction of sentences above a similarity threshold as the semantic redundancy ratio $\rho$.
4. Visualize how $\rho$ changes as we inject irrelevant or contradictory evidence, mirroring the prompt-noise heatmap with real text.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import pandas as pd

plt.style.use("seaborn-v0_8")



In [ ]:
QA_DATA = [
    {
        "question": "What is the capital of Spain?",
        "answer": "Madrid",
        "prompts": {
            "direct": "Question: What is the capital of Spain?",
            "relevant": "Spain is bordered by France and Portugal. The capital city of Spain is Madrid.",
            "noisy": "Spain shares borders with Portugal. Soccer is popular there. Some people mistakenly cite Barcelona as the capital.",
            "contradictory": "Spain's capital used to be Toledo. Many tourists believe Barcelona is the capital today.",
        },
    },
    {
        "question": "Who developed the theory of relativity?",
        "answer": "Albert Einstein",
        "prompts": {
            "direct": "Who developed the theory of relativity?",
            "relevant": "Physics history highlights Albert Einstein, who formulated the theory of relativity.",
            "noisy": "Relativity is discussed alongside quantum mechanics. Isaac Newton studied gravity.",
            "contradictory": "Some blogs claim Nikola Tesla created relativity. Others credit Einstein.",
        },
    },
    {
        "question": "Which element has the chemical symbol O?",
        "answer": "Oxygen",
        "prompts": {
            "direct": "Which element has the chemical symbol O?",
            "relevant": "Oxygen, symbol O, is essential for respiration.",
            "noisy": "The periodic table lists elements like hydrogen and carbon.",
            "contradictory": "Some sources confuse oxygen with osmium because both start with 'O'.",
        },
    },
]



In [ ]:
all_sentences = []
for sample in QA_DATA:
    all_sentences.append(sample["answer"])
    for prompt_text in sample["prompts"].values():
        all_sentences.extend([s.strip() for s in prompt_text.split(".") if s.strip()])

vectorizer = TfidfVectorizer().fit(all_sentences)


def sentence_embedding(text: str) -> np.ndarray:
    vec = vectorizer.transform([text])
    return normalize(vec).toarray().squeeze(0)


def redundancy_ratio(prompt: str, answer_emb: np.ndarray, threshold: float = 0.3) -> float:
    sentences = [s.strip() for s in prompt.split(".") if s.strip()]
    if not sentences:
        return 0.0, np.array([])
    sims = []
    for sentence in sentences:
        emb = sentence_embedding(sentence)
        sim = float(np.dot(emb, answer_emb))
        sims.append(sim)
    sims = np.array(sims)
    rho = np.mean(sims >= threshold)
    return float(rho), sims



In [ ]:
threshold = 0.35
results = []
for sample in QA_DATA:
    answer_emb = sentence_embedding(sample["answer"])
    for label, prompt in sample["prompts"].items():
        rho, sims = redundancy_ratio(prompt, answer_emb, threshold)
        results.append(
            {
                "question": sample["question"],
                "variant": label,
                "rho": rho,
                "avg_similarity": float(np.mean(sims)) if len(sims) else 0.0,
                "sentences": len([s for s in prompt.split(".") if s.strip()]),
            }
        )

results



In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df_pivot = df.pivot(index="question", columns="variant", values="rho")
df_pivot


In [ ]:
plt.figure(figsize=(8, 4))
sns.heatmap(df_pivot.loc[:, ["direct", "relevant", "noisy", "contradictory"]], annot=True, cmap="mako", vmin=0, vmax=1)
plt.title(f"Semantic redundancy ratio (threshold={threshold})")
plt.ylabel("Question")
plt.xlabel("Prompt variant")
plt.show()



Higher $\rho$ values correspond to prompts that contain more sentences aligned with the answer. Direct and relevant variants stay above 0.5, while noisy/contradictory versions drop sharply, matching the intuition from the synthetic heatmap.


In [ ]:
df_summary = df.groupby("variant").agg({"rho": "mean", "avg_similarity": "mean", "sentences": "mean"}).sort_values("rho", ascending=False)
df_summary


We treat the average cosine similarity against the answer as a crude mutual-information proxy. Even with only three QA examples, the ordering is clear: `relevant > direct > noisy > contradictory`, reinforcing the semantic redundancy ratio story.
